Dados: Kaggle Titanic dataset

In [117]:
import numpy as np
import pandas as pd
import plotly.express as px

df = pd.concat([
    pd.read_csv('data/titanic/train.csv'),
    pd.read_csv('data/titanic/test.csv')
])
df['FamiilySize'] = df['SibSp'] + df['Parch']
df['Surname'] = df['Name'].apply(lambda x: x.split(',', 1)[0])
df['Embarked'] = df['Embarked'].astype('category')
df['Survived'] = df['Survived'].astype(bool)
df.drop(columns=['PassengerId', 'Ticket'], inplace=True)
display(df.head())
df.info()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,FamiilySize,Surname
0,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,1,Braund
1,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,1,Cumings
2,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,0,Heikkinen
3,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,1,Futrelle
4,False,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,0,Allen


<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Survived     1309 non-null   bool    
 1   Pclass       1309 non-null   int64   
 2   Name         1309 non-null   object  
 3   Sex          1309 non-null   object  
 4   Age          1046 non-null   float64 
 5   SibSp        1309 non-null   int64   
 6   Parch        1309 non-null   int64   
 7   Fare         1308 non-null   float64 
 8   Cabin        295 non-null    object  
 9   Embarked     1307 non-null   category
 10  FamiilySize  1309 non-null   int64   
 11  Surname      1309 non-null   object  
dtypes: bool(1), category(1), float64(2), int64(4), object(4)
memory usage: 115.2+ KB


## 1- Baseado na descrição do conjunto de dados formule algumas hipótese, perguntas que você acha que podem ser respondidas/entendidas com este conjunto de dados. Tente formular até 3 questões. Numere cada questão.

1. Passageiros com maiores condições econômicas tiveram uma maior taxa de sobrevivência?
2. Quais os fatores mais influentes para o valor da tarifa?
3. Existe relação entre o sobrenome da família com a taxa de sobrevivência?

## 2- Escolha 5 atributos que você acha representar bem o problema e faça uma análise dos tipos de atributos (calcule a matriz de correlação). Justifique a escolha desses atributos no contexto do conjunto de dados e com relação às hipótese/questões levantadas na pergunta 1.

### Atributos escolhidos:
- `Survived`: Atributo alvo;
- `Pclass`: Classe do passageiro;
- `Fare`: Valor da tarifa paga, forte indicativo de uma melhor classe, mas pode ser influenciado por outros fatores, como local de embarque e tamanho da família;
- `Age`: Idade, pessoas mais velhas podem estar mais associadas a classe econômica alta;
- `FamilyNumber`: Tamanho da família também pode ser um indicativo de condição econômica.

In [97]:
data = df[['Survived', 'Pclass', 'Fare', 'Age', 'FamiilySize']].copy()
# Onehot encoding do atributo Pclass
data = pd.concat([
    data,
    pd.get_dummies(data['Pclass'], prefix='Pclass').astype(float),
], axis=1)
display(data.head())
data.info()

,Survived,Pclass,Fare,Age,FamiilySize,Pclass_1,Pclass_2,Pclass_3
0,False,3,7.2500,22.0,1,0.0,0.0,1.0
1,True,1,71.2833,38.0,1,1.0,0.0,0.0
2,True,3,7.9250,26.0,0,0.0,0.0,1.0
3,True,1,53.1000,35.0,1,1.0,0.0,0.0
4,False,3,8.0500,35.0,0,0.0,0.0,1.0


<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Survived     1309 non-null   bool   
 1   Pclass       1309 non-null   int64  
 2   Fare         1308 non-null   float64
 3   Age          1046 non-null   float64
 4   FamiilySize  1309 non-null   int64  
 5   Pclass_1     1309 non-null   float64
 6   Pclass_2     1309 non-null   float64
 7   Pclass_3     1309 non-null   float64
dtypes: bool(1), float64(5), int64(2)
memory usage: 83.1 KB


In [98]:
# Matrix de correlação
data.drop(columns='Pclass').corr()

,Survived,Fare,Age,FamiilySize,Pclass_1,Pclass_2,Pclass_3
Survived,1.000000,0.183737,-0.042698,0.000245,0.199180,0.072678,-0.231918
Fare,0.183737,1.000000,0.178740,0.226492,0.600031,-0.121384,-0.419752
Age,-0.042698,0.178740,1.000000,-0.240229,0.393198,-0.014986,-0.337069
FamiilySize,0.000245,0.226492,-0.240229,1.000000,-0.029656,-0.039976,0.058430
Pclass_1,0.199180,0.600031,0.393198,-0.029656,1.000000,-0.296526,-0.622172
Pclass_2,0.072678,-0.121384,-0.014986,-0.039976,-0.296526,1.000000,-0.563180
Pclass_3,-0.231918,-0.419752,-0.337069,0.058430,-0.622172,-0.563180,1.000000


## 3 - Exiba o mapa de calor (heatmap) correspondente da matriz de correlação.

In [99]:
# Heatmap da matrix de correlação
px.imshow(
    data.drop(columns='Pclass').corr(),
    color_continuous_scale='Viridis',
    title='Heatmap da matriz de correlação',
    width=500,
    height=500)

## 4 - Faça uma análise exploratória com as medidas vistas em aula (média, mediana, moda, obliquidade e kurtose) nos atributos anteriormente selecionados. Explique quais medidas ajudam no entendimento do conjunto de dados. Sinta-se livre para utilizar outras medidas, mas explique porque você está usando elas. Se achar necessário, pode utilizar mais atributos na análise.

In [100]:
print('Estatísticas descritiva dos atributos:')
for col in data.drop(columns=['Pclass_1', 'Pclass_2', 'Pclass_3']).columns:
    # Title with cyan color
    print(f'\n\033[96m{col}\033[0m')
    print(f'Média: {data[col].mean():.2f}')
    print(f'Mediana: {data[col].median():.2f}')
    print(f'Moda: {data[col].mode().values[0]}')
    print(f'Obliquidade: {data[col].skew():.2f}')
    print(f'Kurtose: {data[col].kurt():.2f}')

Estatísticas descritiva dos atributos:

Survived
Média: 0.58
Mediana: 1.00
Moda: True
Obliquidade: -0.33
Kurtose: -1.90

Pclass
Média: 2.29
Mediana: 3.00
Moda: 3
Obliquidade: -0.60
Kurtose: -1.32

Fare
Média: 33.30
Mediana: 14.45
Moda: 8.05
Obliquidade: 4.37
Kurtose: 27.03

Age
Média: 29.88
Mediana: 28.00
Moda: 24.0
Obliquidade: 0.41
Kurtose: 0.15

FamiilySize
Média: 0.88
Mediana: 0.00
Moda: 0
Obliquidade: 2.85
Kurtose: 10.30


- Para a questão 1, a média e a mediana da classe do passageiro (`Pclass`) podem indicar que a maioria dos passageiros seja de uma classe econômica mais alta. A baixa obliquidade e kurtose mostram uma distribuição mais uniforme.
- Para a questão 2, o atributo `Fare` tem uma alta obliquidade (4.37) e curtose (27.03), indicando que a distribuição da tarifa é fortemente inclinada para a direita com caudas pesadas, além de uma correlação média com a primeira classe. Entretando há uma fraca correlação deste atributo com a sobrevivência.

## 5 - Exiba o histograma correspondente a cada um dos atributos escolhidos.

In [104]:
# Histograma dos atributos
for col in data.drop(columns=['Pclass_1', 'Pclass_2', 'Pclass_3']).columns:
    fig = px.histogram(
        data,
        x=col,
        title=f'{col} histogram',
        width=500,
        height=400
    )
    fig.show()

## 6 - Faça o boxplot correspondente a cada um dos atributos escolhidos.

In [106]:
# Boxplot dos atributos
for col in data.drop(columns=[
    'Survived', 'Pclass_1', 'Pclass_2', 'Pclass_3']
).columns:
    fig = px.box(
        data,
        y=col,
        title=f'{col} boxplot',
        width=500,
        height=400
    )
    fig.show()

## 7 - Detectar se existem outliers. Se existirem, quais são eles?

Segundo a visualização dos boxplots, os atributos `Fare` e `Age` possuem outliers.

In [115]:
def iqr_outliers(df: pd.DataFrame, column: str) -> pd.Series:
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_limit = q1 - 1.5 * iqr
    upper_limit = q3 + 1.5 * iqr
    iqr_mask = (df[column] < lower_limit) | (df[column] > upper_limit)
    outliers = df.loc[iqr_mask, column]
    return outliers if not outliers.empty else None

outliers = {key: iqr_outliers(data, key) for key in ['Pclass', 'Fare', 'Age', 'FamiilySize']}
for feature in outliers:
    if outliers[feature] is not None:
        print(f'Quantidade de outliers em {feature}: {outliers[feature].shape[0]}')

Quantidade de outliers em Fare: 171
Quantidade de outliers em Age: 9
Quantidade de outliers em FamiilySize: 125


## 8 - Após a análise exploratória reanalise as hipóteses/questões e comente se elas já podem ser respondidas, se elas não fazem mais sentido, eventualmente se elas podem ser reformuladas para questões

### Questão 1: Passageiros com maiores condições econômicas tiveram uma maior taxa de sobrevivência?

A análise exploratória mostrou que quanto melhor a classe do passageiro (`Pclass`), aumenta a correlação com a taxa de sobrevivência, assim como a tarifa paga (`Fare`), entretando o valor ainda é baixo, necessitando testes estatísticos para confirmar a hipótese. A idade (`Age`) e o tamanho da família (`FamilySize`) não apresentaram correlação significativa com a sobrevivência.

### Questão 2: Quais os fatores mais influentes para o valor da tarifa?

O valor da tarifa tem uma boa correlação com a classe do passageiro (`Pclass`), já a idade (`Age`) e o tamanho da família (`FamilySize`) necessitam de mais análises para verificar se também influenciam.

### Questão 3: Existe relação entre o sobrenome da família com a taxa de sobrevivência?

O atributo sobrenome da família (`Surname`) necessita de uma análise estatística mais específica para verificar se há relação com a taxa de sobrevivência.

In [128]:
import scipy.stats as stats

groups = df.groupby('Surname')['Survived'].apply(list)

# Perform ANOVA test
f_statistic, p_value = stats.f_oneway(*groups)

print('F statistic:', f_statistic)
print('P-value:', p_value)

print('Há diferença significativa entre os grupos de sobrenomes e a taxa de sobrevivência.')

F statistic: 1.3620203024582798
P-value: 0.00013620225134228936
Há diferença significativa entre os grupos de sobrenomes e a taxa de sobrevivência.
